In [ ]:
lap_df = pd.read_pickle('/media/sj-archimedes/data/share/OddAI_Library_practice/data08/lap_trainval_20221001-20231031.pkl')
lap_df = lap_df.query('creative_type == "image"')
lap_df['creative_media_url'] = lap_df['creative_media_url'].map(lambda x: x[0])
lap_df['creative_media_hash'] = lap_df['creative_media_hash'].map(lambda x: x[0])
df = lap_df

all_caption_dict = {}

caption_dict_list = glob('/media/sj-archimedes/data/masaya_kondo/research/mllm/caption/llava1.5-7b_caption/*.pkl')
caption_dict = {}
for fname in caption_dict_list:
    with open(fname, 'rb') as r:
        caption_dict |= pickle.load(r)
all_caption_dict['caption'] = caption_dict

appeal_dict_list = glob('/media/sj-archimedes/data/masaya_kondo/research/mllm/caption/llava1.5-7b_appeal_caption/*.pkl')
appeal_dict = {}
for fname in appeal_dict_list:
    with open(fname, 'rb') as r:
        appeal_dict |= pickle.load(r)
all_caption_dict['appeal'] = appeal_dict

detr_dict_list = glob('/media/sj-archimedes/data/masaya_kondo/research/mllm/caption/detr_blip2_caption/*.pkl')
detr_dict = {}
for fname in detr_dict_list:
    with open(fname, 'rb') as r:
        detr_dict |= pickle.load(r)
detr_dict = {k:'. '.join(v) for k, v in detr_dict.items()}
all_caption_dict['detr_blip2'] = detr_dict

df['caption'] = df['creative_media_hash'].map(
    lambda x: all_caption_dict['caption'][x] if x in all_caption_dict['caption'] else None
)
df['appeal'] = df['creative_media_hash'].map(
    lambda x: all_caption_dict['appeal'][x] if x in all_caption_dict['appeal'] else None
)
df['detr_blip2'] = df['creative_media_hash'].map(
    lambda x: all_caption_dict['detr_blip2'][x] if x in all_caption_dict['detr_blip2'] else None
)
df = df[~df['caption'].isna()]

df = df[['creative_media_hash', 'creative_media_url', 'caption', 'appeal', 'detr_blip2']]
df = df.drop_duplicates()

# df['embeddings_caption'] = df['caption'].progress_apply(
#     lambda x: _ctx_encoder(**_ctx_tokenizer(x, return_tensors='pt', max_length=512).to(device))[0][0].cpu().numpy()
# )
# df['embeddings_appeal'] = df['appeal'].progress_apply(
#     lambda x: _ctx_encoder(**_ctx_tokenizer(x, return_tensors='pt', max_length=512).to(device))[0][0].cpu().numpy()
# )
# df['embeddings_detr_blip2'] = df['detr_blip2'].progress_apply(
#     lambda x: _ctx_encoder(**_ctx_tokenizer(x, return_tensors='pt', max_length=512).to(device))[0][0].cpu().numpy()
# )

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

device = 'cuda:0'
def load_query_model():
    ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
    q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
    ctx_encoder.to(device)
    return ctx_encoder, ctx_tokenizer, q_encoder, q_tokenizer

ctx_encoder, ctx_tokenizer, q_encoder, q_tokenizer = load_query_model()

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [ ]:
loader = DataLoader(dataset, batch_size=16)

all_outputs = []
for batch in tqdm(loader):
    inputs = ctx_tokenizer(batch['caption'], padding=True, truncation=True, max_length=512, return_tensors='pt')
    inputs = {k:v.to(device) for k,v in inputs.items()}
    outputs = ctx_encoder(**inputs)
    outputs = outputs.pooler_output.detach().cpu().tolist()
    all_outputs += outputs

In [ ]:
import numpy as np
df['hoge'] = all_outputs

In [ ]:
# 共通のimport
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import requests
import pandas as pd
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
from PIL import Image, ImageDraw
import pickle
import torch
from transformers import BitsAndBytesConfig
from transformers import pipeline
from tqdm import tqdm
import openai
from openai import OpenAI
from src.utils import download_image_from_s3

import torch
from transformers import AutoTokenizer, AutoModelForVision2Seq, AutoImageProcessor
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from layout_detr.src.layout_detr import LayoutDetrFeatureExtractor
from layout_detr.src.layout_detr import LayoutDetrForObjectDetection

from PIL import Image
import requests

with open('./sample_image/sample_image_dict.pkl', 'rb') as r:
    sample_image_dict = pickle.load(r)

In [ ]:
for genre, img in sample_image_dict.items():
    print(genre)
    fig, ax = plt.subplots(1,3,figsize=(15,5))
    for i in range(3):
        ax[i].imshow(img[i])
    plt.show()

In [ ]:
detor_processor = LayoutDetrFeatureExtractor()
detor_model = LayoutDetrForObjectDetection.from_pretrained('v1-nlabel-1')
detor_model.to('cuda:0')
print()

# BOX検知 調査

In [ ]:
idx = 1
genre = '教育・資格'
img = sample_image_dict[genre][idx].copy()
plt.figure(figsize=(5,5))
plt.imshow(img)
plt.show()


inputs = processor.preprocess(img, return_tensors='pt')
inputs = {k:v.to(detor_model.device) for k, v in inputs.items()}
outputs = detor_model(**inputs)
target_size = torch.Tensor([img.size[::-1]])  # 縦横反転
postprocessed = processor.post_process_object_detection(outputs, target_sizes=target_size, threshold=0)[0]
scores = postprocessed['scores'].cpu().detach().numpy()
boxes = postprocessed['boxes'].cpu().detach().numpy()
N_BOXES = 10
ids = scores.argsort()[::-1][:N_BOXES]
scores = scores[ids]
boxes = boxes[ids]
drawer = ImageDraw.Draw(img)
for box in boxes:
    drawer.rectangle(box, outline='red', width=3)
plt.figure(figsize=(5,5))
plt.imshow(img)
plt.show()

img = sample_image_dict[genre][idx].copy()
num_ax = math.ceil(len(boxes)/5)
plt.subplots(num_ax, 5)
for i, box in enumerate(boxes):
    plt.subplot(num_ax, 5, i+1)
    plt.imshow(img.crop(box))
plt.show()

# BOX 取得
- サンプルデータ的にはBOXの取得数は10がよさそう

In [ ]:
sample_box_image_dict = {}
for genre, image_list in sample_image_dict.items():
    for i, img in enumerate(image_list):
        inputs = detor_processor.preprocess(img, return_tensors='pt')
        inputs = {k:v.to(detor_model.device) for k, v in inputs.items()}
        outputs = detor_model(**inputs)
        target_size = torch.Tensor([img.size[::-1]])  # 縦横反転
        postprocessed = detor_processor.post_process_object_detection(outputs, target_sizes=target_size, threshold=0)[0]
        scores = postprocessed['scores'].cpu().detach().numpy()
        boxes = postprocessed['boxes'].cpu().detach().numpy()
        N_BOXES = 10
        ids = scores.argsort()[::-1][:N_BOXES]  # (１つの式にスライスとストライドを同時に使わない. by Effective Python)
        # ids = ids[:6]
        scores = scores[ids]
        boxes = boxes[ids]
        box_images = [img.crop(box) for box in boxes]
        sample_box_image_dict[f'{genre}_{i}'] = box_images

# BLIP-2

In [ ]:
model_id = '/media/sj-archimedes/data/03_pretrained_model/llm/salesforce/blip2-opt-2.7b'
processor = AutoProcessor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, device_map='auto')

In [ ]:
def blip2_caption(image_list, prompt=None):
    num_images = len(image_list)
    fig, ax = plt.subplots(1, num_images,figsize=(20,5))
    for i in range(num_images):
        ax[i].imshow(image_list[i])
    plt.show()

    caption_result = []
    for img in image_list:
        # 入力の準備
        inputs = processor(img, text=prompt, return_tensors="pt").to(model.device)
        generated_ids = model.generate(**inputs, max_new_tokens=128)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        caption_result.append(generated_text)
    return caption_result

In [ ]:
for genre, image_list in sample_box_image_dict.items():
    caption_list = blip2_caption(image_list)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

In [ ]:
prompt = "Question: Is this a character (or letter)? .Answer:"

for genre, image_list in sample_box_image_dict.items():
    caption_list = blip2_caption(image_list, prompt=prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

# japanese_stable_vlm

In [ ]:
model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/stabilityai/japanese-stable-vlm"
model_kwargs = {"trust_remote_code": True, "low_cpu_mem_usage": True}
model_kwargs["variant"] = "fp16"
model_kwargs["torch_dtype"] = torch.float16
model_kwargs["device_map"] = "auto"

model = AutoModelForVision2Seq.from_pretrained(model_id, **model_kwargs)
processor = AutoImageProcessor.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = model.eval()

In [ ]:
def stable_vlm_caption(image_list, prompt):
    num_images = len(image_list)
    fig, ax = plt.subplots(1, num_images,figsize=(20,5))
    for i in range(num_images):
        ax[i].imshow(image_list[i])
    plt.show()

    caption_result = []
    for img in image_list:
        # 入力の準備
        inputs = processor(images=img, return_tensors="pt")
        text_encoding = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
        inputs.update(text_encoding)
        
        # 推論の実行
        outputs = model.generate(
            **inputs.to(device=model.device),
            do_sample=True,
            num_beams=1,
            max_new_tokens=512,
            min_length=1,
            repetition_penalty=1.5,
            pad_token_id=tokenizer.eos_token_id
        )
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        caption_result.append(generated_text)
    return caption_result

In [ ]:
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
画像を詳細に述べてください。

### 応答: 
"""

for genre, img in sample_box_image_dict.items():
    caption_list = stable_vlm_caption(img, prompt)
    for caption in caption_list:
        print(caption)
        print('\n=====\n')

# LLaVA

In [ ]:
openai.api_key = os.environ['OPENAI_API_KEY']

model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/llava-hf/llava-1.5-7b-hf"

model = pipeline(
    "image-to-text",
    model=model_id,
    device_map='auto',
)

In [ ]:
def llava_caption(image_list, prompt, chatgpt=True):
    num_images = len(image_list)
    fig, ax = plt.subplots(1, num_images,figsize=(20,5))
    for i in range(num_images):
        ax[i].imshow(image_list[i])
    plt.show()

    for img in image_list:
        outputs = model(
            img,
            prompt=prompt,
            generate_kwargs={"max_new_tokens": 512}
        )
        
        response = outputs[0]['generated_text'].split('\nASSISTANT: ')[1]
        
        print(response)

        if chatgpt:
            instruction = f"次の文章を日本語に翻訳してください。\n{response}"
            
            
            client = OpenAI()
            
            response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": instruction},
              ]
            )
            
            response = response.choices[0].message.content
            
            print(response)
        print('\n==========\n')

In [ ]:
# prompt = "USER: <image>\nPlease explain in detail how this advertisement image has been designed to appear attractive to consumers.\nASSISTANT:"
prompt = "USER: <image>\nPlease describe this image.\nASSISTANT:"
for genre, image_list in sample_box_image_dict.items():
    llava_caption(image_list, prompt, chatgpt=False)

In [ ]:
# prompt = "USER: <image>\nPlease explain in detail how this advertisement image has been designed to appear attractive to consumers.\nASSISTANT:"
prompt = """USER: <image>
This image is part of an advertisement. Please answer the following questions.
1. Is it a text layer?
2. What is written on it?
3. Are there any products or items placed?
4. Are there any people?
5. What is depicted in the image?"
ASSISTANT:"""
for genre, image_list in sample_box_image_dict.items():
    llava_caption(image_list, prompt, chatgpt=False)

# スクリプト実装用

In [ ]:
output_dir = "./output_detr_blip2_caption"
target_medias = ['lap', 'fba']

detor_processor = LayoutDetrFeatureExtractor()
detor_model = LayoutDetrForObjectDetection.from_pretrained('v1-nlabel-1')
detor_model.to('cuda:0')
print()


model_id = '/media/sj-archimedes/data/03_pretrained_model/llm/salesforce/blip2-opt-2.7b'
processor = AutoProcessor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, device_map='auto')


def blip2_caption(image_list, prompt=None):
    num_images = len(image_list)

    caption_result = []
    for img in image_list:
        # 入力の準備
        inputs = processor(img, text=prompt, return_tensors="pt").to(model.device)
        generated_ids = model.generate(**inputs, max_new_tokens=128)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        caption_result.append(generated_text)
    return caption_result

for media in target_medias:
    print(media)
    df = pd.read_pickle(f'/media/sj-archimedes/data/share/OddAI_Library_practice/data08/{media}_trainval_20221001-20231031.pkl')
    df = df.query('creative_type == "image"')
    df['creative_media_url'] = df['creative_media_url'].map(lambda x: x[0])
    df['creative_media_hash'] = df['creative_media_hash'].map(lambda x: x[0])
    df = df[['creative_media_url', 'creative_media_hash']]
    df = df.drop_duplicates()

    caption_dict = {}
    for i, row in tqdm(enumerate(df.itertuples()), total=df.shape[0]):
        s3_url = row.creative_media_url
        media_hash = row.creative_media_hash
        if media_hash in caption_dict:
            continue
        img = download_image_from_s3(s3_url)
        inputs = detor_processor.preprocess(img, return_tensors='pt')
        inputs = {k:v.to(detor_model.device) for k, v in inputs.items()}
        outputs = detor_model(**inputs)
        target_size = torch.Tensor([img.size[::-1]])  # 縦横反転
        postprocessed = detor_processor.post_process_object_detection(outputs, target_sizes=target_size, threshold=0)[0]
        scores = postprocessed['scores'].cpu().detach().numpy()
        boxes = postprocessed['boxes'].cpu().detach().numpy()
        N_BOXES = 10
        ids = scores.argsort()[::-1][:N_BOXES]  # (１つの式にスライスとストライドを同時に使わない. by Effective Python)
        scores = scores[ids]
        boxes = boxes[ids]
        box_images = [img.crop(box) for box in boxes]
        response = blip2_caption(box_images, prompt=None)

        caption_dict[media_hash] = response

        if len(caption_dict) == 10:
            with open(f"{output_dir}/{media}_{i}.pkl", "wb") as w:
                pickle.dump(caption_dict, w)
            caption_dict = {}

In [ ]:
import pickle
with open('./output_detr_blip2_caption/lap_19.pkl', 'rb') as r:
    outputs = pickle.load(r)

In [ ]:
outputs